# 0.数据库基础概念
**DDL**: data definition language 数据定义语言，**对整个数据库和表操作，不涉及数据**
- 对数据库的常用操作
- 对表结构的常用操作
- 修改表结构



**DML**:data manipulation language数据库操作语言
对数据库记录进行更新：插入(insert)、删除(delete)、更新(update)

# 1. 执行顺序
|关键字|解释|
|----|:----|
|from|	首先对from子句中的前两个表执行一个笛卡尔乘积，此时生成虚拟表 vt1（选择相对小的表做基础表）|
|on |	接下来便是应用on筛选器，on 中的逻辑表达式将应用到 vt1 中的各个行，筛选出满足on逻辑表达式的行，生成虚拟表 vt2 
|join |	如果是outer join 那么这一步就将添加外部行，left outer jion 就把左表在第二步中过滤的添加进来，如果是right outer join 那么就将右表在第二步中过滤掉的行添加进来，这样生成虚拟表 vt3
|	|如果 from 子句中的表数目多余两个表，那么就将vt3和第三个表连接从而计算笛卡尔乘积，生成虚拟表，该过程就是一个重复1-3的步骤，最终得到一个新的虚拟表 vt3|
|where|	应用where筛选器，对上一步生产的虚拟表引用where筛选器，生成虚拟表vt4注意where与on的区别：先执行on，后执行where；on是建立关联关系在生成临时表时候执行，where是在临时表生成后对数据进行筛选的|
|group by|(select别名可以使用)	group by 子句将中的唯一的值组合成为一组，得到虚拟表vt5。如果应用了group by，那么后面的所有步骤都只能得到的vt5的列或者是聚合函数（count、sum、avg等）。原因在于最终的结果集中只为每个组包含一行。这一点请牢记|
|avg,sum|	应用avg或者sum选项，为vt5生成超组，生成vt6|
|having| 	应用having筛选器，生成vt7,having筛选器是第一个也是为唯一一个应用到已分组数据的筛选器。 
|select|	处理select子句。将vt7中的在select中出现的列筛选出来。生成vt8|
|distinct| 	应用distinct子句，对vt8进行去重，生成vt9|
|order by|	应用order by子句。按照order_by_condition排序vt9，此时返回的一个游标，而不是虚拟表。|
|limit	|应用limit选项。生成vt10返回结果给请求者即用户|

# 2.Mysql约束
- 主键约束(primary key)PK
    - 一个列或多个列(联合主键，constraint primary key(id,name))，唯一标识每一行，尽快查找数据
    - 唯一，非空
    - 最多只允许一个主键
    - 建立对应的唯一索引
|操作|语句|
|:----|:-----|
|添加主键（修改表结构）|	Alter table name add primary key(id,name)|
|删除主键|	Alter table name drop primary key|
- 自增长约束(auto_increment)
    - 主键列实现自增长：初始值1
    - 字段只能是非空，整数
    - 括号后面 Auto_increment=指定初始值
    - delete数据之后自动增长从断点开始；truncate数据之后自动增长从默认起始值开始
- 非空约束(not null)
|操作|语句|
|:----|:-----|
|添加|Alter table name modify id int not null|
|删除|Alter table name modify id int|
- 唯一性约束(unique)
    - 创建表时在字段数据类型后加unique
    - Alter table name add constraint 约束名 unique(列)
    - 可以为多个null，每个null都不一样
    - Alter table name drop index 约束名(若约束没有名字，默认情况为列名)
- 默认约束(default)
- 零填充约束(zerofill)

# 3.Mysql常用操作表

## 3.1 权限操作
|操作|语句|&nbsp;&nbsp;注意点|
|:---|:---|:---|
|登录|	`mysql -u root -p`| |
|查看用户，主机|	`SELECT User, Host FROM mysql.user;`||
|创建用户|	`CREATE USER 'someuser'@'localhost' IDENTIFIED BY 'somepassword';`||
|赋权限所有数据库权限给某用户|	`GRANT ALL PRIVILEGES ON * . * TO 'someuser'@'localhost';`<br/>`FLUSH PRIVILEGES;`||
|查看某用户权限|	`SHOW GRANTS FOR 'someuser'@'localhost';`||
|删除某用户所有权限|`	REVOKE ALL PRIVILEGES, GRANT OPTION FROM 'someuser'@'localhost';`||
|删除用户|	`DROP USER 'someuser'@'localhost';`||
|退出|	`exit;`||

## 3.2 DDL数据库、表定义语言
|操作&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|语句|注意点|
|:---|:---|:---|
|查看数据库|	`SHOW DATABASES;`||
|创建数据库|	`CREATE DATABASE acme;`||
|删除数据库|	`DROP DATABASE acme;`||
|选择数据库|	`USE acme;`||
|创建表|	`CREATE TABLE users(id INT AUTO_INCREMENT, first_name VARCHAR(100),last_name VARCHAR(100), email VARCHAR(50),password VARCHAR(20), location VARCHAR(100),dept VARCHAR(100), is_admin TINYINT(1), register_date DATETIME, PRIMARY KEY(id));`||
|删除表|	`DROP TABLE tablename;`||
|查看表|`SHOW TABLES;`|查看整个数据库中包含的表格|
|新增列|`ALTER TABLE users ADD age VARCHAR(3);`<br />`select id,case when p_id is null then 'Root'  when id in (select p_id from tree ) then 'Inner' else 'Leaf' end as Type from tree `||
|修改列|`ALTER TABLE users MODIFY COLUMN age INT(3);`||

## 3.3DML数据库操作语言
|操作&nbsp;&nbsp;&nbsp;|语句|注意点&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|:---|:---|:---|
|新增记录/行|	`INSERT INTO users (first_name, last_name, email, password, location, dept, is_admin, register_date) values ('Brad', 'Traversy', 'brad@gmail.com', '123456','Massachusetts', 'development', 1, now());`||
|新增多行|	`INSERT INTO users (first_name, last_name, email, password, location, dept,  is_admin, register_date) values ('Fred', 'Smith', 'fred@gmail.com', '123456', 'New York', 'design', 0, now()), ('Sara', 'Watson', 'sara@gmail.com', '123456', 'New York', 'design', 0, now()),('Will', 'Jackson', 'will@yahoo.com', '123456', 'Rhode Island', 'development', 1, now()),('Paula', 'Johnson', 'paula@yahoo.com', '123456', 'Massachusetts', 'sales', 0, now()),('Tom', 'Spears', 'tom@yahoo.com', '123456', 'Massachusetts', 'sales', 0, now());`||
|**查询**|	`SELECT * FROM users;`<br />`SELECT first_name, last_name FROM users;`||
|条件查询|`SELECT * FROM users WHERE location='Massachusetts';`<br/>`SELECT * FROM users WHERE location='Massachusetts' AND dept='sales';`<br />`SELECT * FROM users WHERE is_admin = 1;`<br />`SELECT * FROM users WHERE is_admin > 0;`||
|模糊查询 Like（全表扫描，速度慢）|`SELECT * FROM users WHERE dept LIKE 'd%';`<br />`SELECT * FROM users WHERE dept LIKE '%t';`<br />`SELECT * FROM users WHERE dept LIKE '%e%';`|**\_**：匹配任意一个字符；<br />**%**：匹配0个或多个字符;<br />\[ \]：匹配\[ \]中的任意一个字符(若要比较的字符是连续的，则可以用连字符“-”表 达 )；<br />\[^ \]：不匹配\[ \]中的任意一个字符|
|Not like|`SELECT * FROM users WHERE dept NOT LIKE 'd%';`||
|In|`SELECT * FROM users WHERE dept IN ('design', 'sales');`||
|**高级查询**最值|`select max(gpa) from user_profile where university='复旦大学'`|聚合函数|
|limit限制查询|`select gpa from user_profile where university='复旦大学' order by gpa desc limit n`||
|分组查询&聚合函数|`select university,avg(question_cnt) as avg_question_cnt,avg(answer_cnt) as avg_answer_cnt from user_profile group by university having avg_question_cnt <5 or avg_answer_cnt<20`|分组后筛选聚合函数用having,而不是where|
|删除行|`DELETE FROM users WHERE id = 6;`||
||`TRUNCATE users;`|删除整个表，再创建新表|
|按条件更新行|`UPDATE users SET email = 'freddy@gmail.com' WHERE id = 2;`<br/>`UPDATE salary SET sex = CASE sex WHEN 'm' THEN 'f' ELSE 'm'END;`||

## 3.4 多表查询
|操作&nbsp;&nbsp;&nbsp;|语句|注意点&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|:---|:---|:---|
|左连接|`SELECT comments.body,posts.title FROM comments LEFT JOIN posts ON posts.id = comments.post_id ORDER BY posts.title;`||
|连接查询|`select question_practice_detail.device_id,question_id,result from question_practice_detail,user_profile where question_practice_detail.device_id=user_profile.device_id and user_profile.university='浙江大学';`||
|子查询|`select device_id,question_id,result from question_practice_detail where device_id=(select device_id from user_profile where university="浙江大学");`||

|连接方式|连接规则|实现语句|优势|劣势|
|:-------|:-------|:-------|:----|:----|
|内部连接（INNER JOIN）|内部连接只返回两个表中都存在匹配数据的行，如果两个表中没有匹配的记录，则不会返回任何结果|它可以使用 ON 或 USING |准确地过滤出匹配的数据，并且查询效率高|某些表之间的关系是弱化的，那么这种方式会导致查询结果缺失|
|左连接（LEFT OUTER JOIN）|以左表为基础，返回所有左表中的行，同时显示右表中符合条件的行，如果右表中没有符合条件的行，则用 NULL 补充|按照 ON 或 USING 指定的条件连接右表|左连接保留了左表中的所有记录，即使右表中没有匹配的记录|会导致查询结果出现大量的 NULL 值|
|右连接（RIGHT OUTER JOIN）|右连接以右表为基础，回所有右表中的行，同时显示左表中符合条件的行，如果左表中没有符合条件的行，则用 NULL 补充|按照 ON 或 USING 指定的条件连接左表|右连接保留了右表中的所有记录|使用次数相对较少，很多场合都可以通过内部连接或者左连接实现|
